# Dependencies

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

# Train the model

In [4]:
path = "data/dogscats/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [5]:
train_path=path + '/train/'
valid_path=path + '/valid/'

In [6]:
batch_size = 64

In [32]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [40]:
batches = get_batches(train_path, shuffle=True, batch_size=batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

### Drop last fully connected layer

In [34]:
model.pop()
for layer in model.layers: layer.trainable = False

In [35]:
model.add(Dense(2, activation='softmax'))

In [46]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch,
                       validation_data=val_batches, nb_val_samples=val_batches.N)

In [47]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
# First Finetune
fit_model(model, batches, val_batches, nb_epoch=2)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


Epoch 1/2
23000/23000 [==============================] - 780s - loss: 0.6126 - acc: 0.9593 - val_loss: 0.3591 - val_acc: 0.9775
Epoch 2/2
23000/23000 [==============================] - 782s - loss: 0.5406 - acc: 0.9656 - val_loss: 0.2980 - val_acc: 0.9810


In [49]:
model.save_weights(model_path + 'finetune1.h5')

In [50]:
model.load_weights(model_path + 'finetune1.h5')

In [22]:
model.summary()

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [51]:
model.evaluate_generator(val_batches, val_batches.nb_sample)

[0.2980008125305218, 0.98099999999999998]

## Second Finetune

In [52]:
layers = model.layers
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]
for layer in layers[first_dense_idx:]: layer.trainable = True

In [53]:
# K is keras.backend
K.set_value(opt.lr, 0.01)
fit_model(model, batches, val_batches, nb_epoch=3)

Epoch 1/3
23000/23000 [==============================] - 781s - loss: 0.4132 - acc: 0.9738 - val_loss: 0.2786 - val_acc: 0.9825
Epoch 2/3
23000/23000 [==============================] - 782s - loss: 0.3904 - acc: 0.9752 - val_loss: 0.3056 - val_acc: 0.9795
Epoch 3/3
23000/23000 [==============================] - 782s - loss: 0.3999 - acc: 0.9748 - val_loss: 0.2841 - val_acc: 0.9820


In [54]:
model.save_weights(model_path + 'finetune2.h5')

In [25]:
model.load_weights(model_path + 'finetune2.h5')

In [26]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [74]:
model.evaluate_generator(val_batches, val_batches.nb_sample)

[0.28408272589371031, 0.98199999999999998]

In [56]:
??model.evaluate_generator

## Removing Dropout

In [73]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [72]:
model.load_weights(model_path + 'finetune2.h5')

In [75]:
layers = model.layers

In [76]:
last_conv_idx = [index for index, layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [77]:
last_conv_idx

30

In [78]:
layers[last_conv_idx]

In [79]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]

In [80]:
batches = get_batches(train_path, shuffle=False, batch_size=batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [81]:
batches.class_indices

{'cats': 0, 'dogs': 1}

In [62]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


In [63]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [64]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [82]:
trn_features = load_array(model_path + 'train_convlayer_features.bc')
val_features = load_array(model_path + 'valid_convlayer_features.bc')

In [71]:
trn_features.shape

(23000, 512, 14, 14)

In [65]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [83]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [84]:
conv_layers[-1].output_shape

(None, 512, 14, 14)

In [85]:
def get_fc_model():
    model = Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(2, activation='softmax')
        ])
    
    for l1, l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [86]:
fc_model = get_fc_model()

In [87]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
            batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 50s - loss: 0.2683 - acc: 0.9812 - val_loss: 0.1830 - val_acc: 0.9875
Epoch 2/8
23000/23000 [==============================] - 50s - loss: 0.2127 - acc: 0.9849 - val_loss: 0.2449 - val_acc: 0.9820
Epoch 3/8
23000/23000 [==============================] - 50s - loss: 0.1716 - acc: 0.9882 - val_loss: 0.2181 - val_acc: 0.9845
Epoch 4/8
23000/23000 [==============================] - 50s - loss: 0.1451 - acc: 0.9902 - val_loss: 0.2319 - val_acc: 0.9840
Epoch 5/8
23000/23000 [==============================] - 50s - loss: 0.1215 - acc: 0.9919 - val_loss: 0.1976 - val_acc: 0.9860
Epoch 6/8
23000/23000 [==============================] - 50s - loss: 0.1104 - acc: 0.9927 - val_loss: 0.2169 - val_acc: 0.9835
Epoch 7/8
23000/23000 [==============================] - 50s - loss: 0.0942 - acc: 0.9937 - val_loss: 0.2294 - val_acc: 0.9850
Epoch 8/8
23000/23000 [==============================] - 50s -

In [88]:
fc_model.save_weights(model_path + 'no_dropout.h5')

In [ ]:
fc_model.load_weights(model_path + 'no_dropout.h5')